## Evolution of patenting activity in Europe - An analysis of technological fields

In this notebook, we analyze the evolution of patenting activity in Europe, focusing on changes within individual technological fields as defined by the World Intellectual Property Organization (WIPO). Using data from PATSTAT Global, we will extract raw data, process it into a structured format using Pandas, and utilize third-party libraries to visualize the findings in an engaging manner. 

### Prerrequisites
This notebook has been written for an audience already familiar with TIP. For users completely new to the platform, we recommend to follow the [TIP basic course](https://e-courses.epo.org/course/view.php?id=416) before reading this analysis. 

### Disclaimer 
This notebook is published as a showcase of the capabilities of TIP as a data processing environment. It is not intended to derive any statistical conclusion other than the presentation of the retrieved data. The EPO is not expressing any opinion extrapolated from the analysis.

### The WIPO technology fields
WIPO provides a comprehensive technology concordance table that is particularly useful for studying general technology trends. Patent classification schemes such as the International Patent Classification (IPC) can often be too granular for broad statistical analysis. To address this, the WIPO concordance organizes IPC symbols into 35 distinct technology fields, further grouped into five higher-level technology sectors:

- Electrical engineering
- Instruments
- Chemistry
- Mechanical engineering
- Other fields

The PATSTAT Global database links IPC symbols to WIPO technology fields via the `TLS230_APPLN_TECHN_FIELD` table, which serves as the foundation for our analysis.



## Initialization
As with any analysis involving PATSTAT, we need to initialize the client. The PATSTAT client can be initialized in production with access to the whole PATSTAT database, or in test with a smaller test database. This notebook is published with the client initialized in production with `env='PROD'`. If you want to modify it, we recommend you initialze the client with `env='TEST'` to reduce the time to process your queries. When you are happy with your code and are ready to conduct the real analysis, you can change the environment to production.

We will be working with object relational mapping (ORM) as the recommended method of retrieving data from PATSTAT. For that need to import the three PATSTAT tables we will be working with. 

In [1]:
from epo.tipdata.patstat import PatstatClient

# Initialize the PATSTAT client
patstat = PatstatClient(env='PROD')

# Access ORM
db = patstat.orm()

# Importing the necessary tables
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS901_TECHN_FIELD_IPC, TLS230_APPLN_TECHN_FIELD

# Retrieving the data
We are going to get the `application_id`, the year of filing, the technical field number and the weight associated with each technical field, for applications filed up to 2022. For that, we need to join the tables `TLS201_APPLN` and `TLS230_APPLN_TECHN_FIELD`, since table 230 is the one containing the relationship between each application and its technical fields. 

In [2]:
q = db.query(
    TLS201_APPLN.appln_id,
    TLS201_APPLN.appln_filing_year.label('Year'),
    TLS230_APPLN_TECHN_FIELD.techn_field_nr,
    TLS230_APPLN_TECHN_FIELD.weight
).join(
    TLS230_APPLN_TECHN_FIELD, TLS201_APPLN.appln_id == TLS230_APPLN_TECHN_FIELD.appln_id
).filter(
    TLS201_APPLN.appln_filing_year <= 2022, 
    TLS201_APPLN.appln_auth == 'EP',
)

res = patstat.df(q)

res

,appln_id,Year,techn_field_nr,weight
0,405364654,2013,22,0.250000
1,546671899,2020,22,0.125000
2,547771719,2020,22,0.142857
3,376821126,2012,22,0.166667
4,364336892,2012,22,0.076923
...,...,...,...,...
6831219,17320425,1998,35,1.000000
6831220,535865597,2019,35,0.111111
6831221,329288306,2010,35,1.000000
6831222,16376053,2007,35,0.666667


## Understanding the query
This query retrieves data on European patent applications, focusing on their filing year, technological field, and associated weights from the **PATSTAT** database.

### `db.query`:
Creates the query object that specifies what columns to retrieve.

- **`TLS201_APPLN.appln_id`**: The unique identifier for each patent application.
- **`TLS201_APPLN.appln_filing_year`**: The year the patent application was filed. This column is labeled as `'Year'` for clarity in the resulting DataFrame.
- **`TLS230_APPLN_TECHN_FIELD.techn_field_nr`**: The code representing the technological field associated with the application.
- **`TLS230_APPLN_TECHN_FIELD.weight`**: A numerical value indicating the relative contribution of this application to the technological field.


### Joining Tables

The query joins two tables:
1. **`TLS201_APPLN`**: Contains general information about patent applications.
2. **`TLS230_APPLN_TECHN_FIELD`**: Contains data linking applications to their respective WIPO technology fields and weights.

The join is based on the `appln_id` column, which is present in both tables and serves as the foreign key linking them.


### Filtering Results
Filters the query results to include only:
1. Applications filed on or before the year 2022 (`TLS201_APPLN.appln_filing_year <= 2022`).
2. Applications with `'EP'` (European Patent Office) as the authority (`TLS201_APPLN.appln_auth == 'EP'`).


### Result

The variable **`res`** holds the resulting DataFrame, which contains the following columns:
- **`appln_id`**: Unique application identifier.
- **`Year`**: Year of application filing.
- **`techn_field_nr`**: Technology field code.
- **`weight`**: Relative weight of the application in its technology field.

Since the query joins the `TLS201_APPLN` and `TLS230_APPLN_TECHN_FIELD` tables, an application (`appln_id`) can appear multiple times in the resulting DataFrame. This occurs because a single application can be associated with multiple WIPO technology fields (`techn_field_nr`). Each record represents one unique combination of an application and its corresponding technology field, along with the relative weight (`weight`) indicating its contribution to that field.


## The weights  in `TLS230_APPLN_TECHN_FIELD`
These weights reflect the proportional contribution of a patent application to a specific WIPO technology field. The weight is calculated by dividing 1 by the number of unique IPC subclasses assigned to the application. If a patent is classified into multiple IPC subclasses, each subclass receives an equal fraction of the total weight. This ensures that each patent contributes equally to the technological fields it is assigned to, avoiding overrepresentation of applications with extensive classification coverage.

For more information about the weights please consult the [data catalogue](https://link.epo.org/web/searching-for-patents/business/patstat/data-catalog-patsat-global-spring-en.pdf).



## Aggregating the weights
The next step for the analysis is to aggregate the weights per year and per technology field. Here it is important to consider that although there can be multiple entries for a given application in our dataframe, the sum of the weights for one given application should be one. It follows that when we sum the weights per field we are not counting applications more than once. What is happening is that the contribution of each application to each technology field for that year is accurately distributed. 


In this analysis, we will use pandas DataFrames, a versatile tool for data manipulation and analysis in Python, allowing efficient operations like filtering, aggregation, and reshaping. Pandas comes preinstalled in TIP so every time you generate a DataFrame like `res` you can use pandas functions like `groupby`.


In [3]:
# Group by 'Year' and 'techn_field_nr' and count the number of 'appln_id'
aggregated = res.groupby(['Year', 'techn_field_nr'])['weight'].sum().reset_index()

# Round the 'weight' column to integers
aggregated['weight'] = aggregated['weight'].round().astype(int)

# Display the aggregated DataFrame
aggregated



,Year,techn_field_nr,weight
0,1978,1,182
1,1978,2,40
2,1978,3,35
3,1978,4,7
4,1978,5,13
...,...,...,...
1566,2022,31,3030
1567,2022,32,6885
1568,2022,33,2188
1569,2022,34,2916


### Understanding the aggregated DataFrame
With the use of `groupby` we now have, for each year in the period 1978-2022, the sum of weights per technical field, for the applications filed on each specific year. For example, in 1978 there was a combined weight of 182 for technical field number one from applications filed that year. In order to have a complete analysis we need to retrieve the actual names of the technology fields, which we will do next. 

## Retrieving the names of the technology fields

The `TLS901_TECH_FIELD_IPC` table provides descriptive names (`techn_field`) for the numerical technology field identifiers (`techn_field_nr`). These names make the analysis more intuitive, as the numerical codes alone are not descriptive enough.

We query PATSTAT Global again to get the technical field names.

In [5]:
q = db.query(
    TLS901_TECHN_FIELD_IPC.techn_field_nr,
    TLS901_TECHN_FIELD_IPC.techn_field,
    TLS901_TECHN_FIELD_IPC.techn_sector       
)
fields  = patstat.df(q)

fields

,techn_field_nr,techn_field,techn_sector
0,1,"Electrical machinery, apparatus, energy",Electrical engineering
1,1,"Electrical machinery, apparatus, energy",Electrical engineering
2,1,"Electrical machinery, apparatus, energy",Electrical engineering
3,1,"Electrical machinery, apparatus, energy",Electrical engineering
4,1,"Electrical machinery, apparatus, energy",Electrical engineering
...,...,...,...
765,35,Civil engineering,Other fields
766,35,Civil engineering,Other fields
767,35,Civil engineering,Other fields
768,35,Civil engineering,Other fields


### Understanding multiple entries for each field

The `TLS901_TECH_FIELD_IPC` table includes not only the technology fields (`techn_field_nr` and `techn_field`) but also the broader sectors (`techn_sector`) to which each field belongs. Since there are multiple fields within each sector, this table contains repeated entries for each field, corresponding to its association with a specific sector. This structure allows for the hierarchical organization of technology. 

We are not interested in the broader sectors, so we use Pandas `drop_duplicates` function to keep one entry for each field. We also drop the column `tech_sector` since we are no longer interested in it. 


In [6]:
unique_fields = fields.drop_duplicates().drop(columns=['techn_sector'])
unique_fields

,techn_field_nr,techn_field
0,1,"Electrical machinery, apparatus, energy"
30,2,Audio-visual technology
48,3,Telecommunications
58,4,Digital communication
61,5,Basic communication processes
71,6,Computer technology
88,7,IT methods for management
89,8,Semiconductors
91,9,Optics
101,10,Measurement


### Merging the two dataframes

In this step, we merge the aggregated weights DataFrame with the unique fields DataFrame. This allows us to replace the numerical `techn_field_nr` with its corresponding descriptive name (`techn_field`). 

The merge is performed using the `techn_field_nr` column as the key, with an inner join to ensure only matching rows from both DataFrames are included in the result.

Alternatively, this merge could also have been achieved by directly joining the `TLS230_APPLN_TECHN_FIELD` table with the `TLS901_TECH_FIELD_IPC` table in the query step. In this notebook we are doing the merging of the dataframes from two different queries to show this posibility of post processing of data and to give a clearer view of the data in each table. 


In [7]:
merged = aggregated.merge(unique_fields, on=['techn_field_nr', 'techn_field_nr'], how='inner')
merged

,Year,techn_field_nr,weight,techn_field
0,1978,1,182,"Electrical machinery, apparatus, energy"
1,1978,2,40,Audio-visual technology
2,1978,3,35,Telecommunications
3,1978,4,7,Digital communication
4,1978,5,13,Basic communication processes
...,...,...,...,...
1566,2022,31,3030,Mechanical elements
1567,2022,32,6885,Transport
1568,2022,33,2188,"Furniture, games"
1569,2022,34,2916,Other consumer goods


## Formatting the merged DataFrame
In this analysis, as discussed above, we are not strictly counting applications, but aggregating the weights of each technology field in each application. However, for storytelling purposes we consider that our stakeholders do not know the details of patstat and we use the accumulated weight for each field and year as a proxy for patenting activity. 

In fact, when running the same analysis by counting the applications only once, considering only the top weight per application, we get very similar numbers. 

Based on the above, we will rename the column `weight` as `Applications` and give the others descriptive names. 

In [8]:
final = merged.drop(columns=['techn_field_nr'])
final = final.rename(columns={'Year': 'Year', 'weight': 'Applications', 'techn_field': 'Field'})
final


,Year,Applications,Field
0,1978,182,"Electrical machinery, apparatus, energy"
1,1978,40,Audio-visual technology
2,1978,35,Telecommunications
3,1978,7,Digital communication
4,1978,13,Basic communication processes
...,...,...,...
1566,2022,3030,Mechanical elements
1567,2022,6885,Transport
1568,2022,2188,"Furniture, games"
1569,2022,2916,Other consumer goods


# Pivoting the dataframe
To make the data more understandable, we pivot the DataFrame so that each technology field becomes a column, with the rows indexed by year. This structure allows for a clearer overview of how patent applications are distributed across different fields over time.We will set the year as the index, which is a pre-requisite for the creation of the video that we will do next. 


In [9]:
### Pivot the DataFrame with 'Year' as the index and 'Field' as the columns
pivoted = final.pivot(index='Year', columns='Field', values='Applications')

# Display the pivoted DataFrame
pivoted


Field,Analysis of biological materials,Audio-visual technology,Basic communication processes,Basic materials chemistry,Biotechnology,Chemical engineering,Civil engineering,Computer technology,Control,Digital communication,...,Organic fine chemistry,Other consumer goods,Other special machines,Pharmaceuticals,Semiconductors,"Surface technology, coating",Telecommunications,Textile and paper machines,Thermal processes and apparatus,Transport
Year,,,,,,,,,,,,,,,,,,,,,
1978,10.0,40.0,13.0,198.0,45.0,121.0,118.0,17.0,54.0,7.0,...,596.0,24.0,147.0,181.0,97.0,46.0,35.0,78.0,76.0,114.0
1979,78.0,271.0,127.0,610.0,115.0,410.0,443.0,218.0,165.0,69.0,...,1315.0,165.0,501.0,399.0,226.0,220.0,181.0,399.0,289.0,411.0
1980,120.0,540.0,292.0,987.0,268.0,793.0,740.0,465.0,295.0,130.0,...,1659.0,370.0,888.0,519.0,456.0,406.0,356.0,727.0,498.0,726.0
1981,159.0,710.0,366.0,1141.0,347.0,952.0,951.0,657.0,396.0,137.0,...,1927.0,516.0,1140.0,684.0,546.0,567.0,467.0,965.0,732.0,1029.0
1982,163.0,951.0,423.0,1227.0,416.0,1002.0,1055.0,734.0,457.0,145.0,...,1999.0,562.0,1368.0,745.0,655.0,541.0,481.0,995.0,639.0,1116.0
1983,219.0,1235.0,495.0,1304.0,546.0,1103.0,1202.0,864.0,505.0,219.0,...,2133.0,660.0,1429.0,921.0,697.0,570.0,606.0,1100.0,638.0,1154.0
1984,260.0,1453.0,579.0,1456.0,850.0,1241.0,1423.0,994.0,592.0,254.0,...,2382.0,746.0,1731.0,1038.0,782.0,751.0,626.0,1309.0,724.0,1363.0
1985,317.0,1573.0,566.0,1401.0,993.0,1380.0,1461.0,1077.0,561.0,288.0,...,2531.0,795.0,1828.0,1287.0,891.0,795.0,729.0,1269.0,720.0,1461.0
1986,338.0,1721.0,684.0,1609.0,1136.0,1524.0,1536.0,1257.0,632.0,306.0,...,2444.0,863.0,1995.0,1460.0,950.0,817.0,844.0,1374.0,731.0,1715.0


## Storytelling with data

One of the advantages of using TIP is the ability to integrate external libraries to enhance data visualization and storytelling. For this notebook, we have chosen to create a **bar chart race video** that dynamically shows the evolution of patent applications across technology fields over time.




### Installing the `bar_chart_race` library 
In TIP, you can install python libraries directly within the notebook using commands like `pip install`. However, it is important to note that any libraries installed during your session will not persist once the session is closed. 

For this reason, it is a good practice to include the installation command for external libraries in the notebook itself. This ensures that anyone running the notebook in the future can install the necessary dependencies without additional effort.

In [ ]:
pip install bar_chart_race --quiet

### Removing odd years to smoothen the transitions in the video

To make the video smoother and more visually appealing, we filtered the data to include only **even years**. This adjustment reduces abrupt transitions while retaining the overall trends in the data resulting in a more visually appealing video. 

The filtering is performed using the condition `pivoted.index % 2 == 0`, which checks if the year (index) is divisible by 2.


In [ ]:
# Filter the DataFrame to include only even years
pivoted_even_years = pivoted[pivoted.index % 2 == 0]

# Display the filtered DataFrame
pivoted_even_years


### Generating the video
We use the `bar_chart_race` to generate a video based on the pivoted DataFrame. The explanation on how to use this library is outside the scope of this notebook. You can get more information on how to use this library in its [official documentation](https://www.dexplo.org/bar_chart_race/). The video will be created and saved in the same folder where this notebook is saved. 

**Note:** The process of generating the video should last a few minutes. The file race.mp4 is generated at the beginning of the process and gets updated by the code below every few seconds. Give the script time to finish before you watch or download the video. There is a warning issued when running the code below. This is to be expected. 

In [ ]:
import bar_chart_race as bcr
bcr.bar_chart_race(
    df=pivoted_even_years,
    filename='race.mp4',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    fixed_max=False,  
    steps_per_period=720,
    period_length=5000,
    interpolate_period=True,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='{x:.0f}',
    cmap='dark12',
    title='Applications per year',
    bar_label_size=7,
    tick_label_size=7,
    scale='linear',
    fig=None,
    writer=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=True
)

## Displaying the video in TIP
With the code above we have created a video `race.mp4` that gets stored in your TIP workspace. If you do not want to save the video but simply visualize it in your notebook you can set `filename = None` in the `bar_chart_race` function. 

The next block of code allows you to display any mp4 video that you have stored on your TIP workspace.

In [ ]:
from IPython.display import Video

# Ensure the correct path to your MP4 file
Video("race.mp4", embed=True)

## Conclusion 
In this notebook we have explored the possibilities of TIP for retrieving data about the technical fields of the European applications. This is nothing new, since you can retrieve the exact same data from PATSTAT online. We did not stop there, though. By using the built-in pandas library, we have combined the data from two queries and processed the combined data to build powerful storytelling visuals. 

Feel free to clone this notebook from its GitHub repository and adjust it to your patent data intelligence needs!